<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# NLP Basics

**Prediction of Sequences of Numbers (RNNs)**

&copy; Dr. Yves J. Hilpisch

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/natural_language_processing.git
import sys
sys.path.append('natural_language_processing')


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
np.set_printoptions(suppress=True)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

## Recurrent Neural Networks

For schematic representations, see e.g. https://certificate.tpq.io/aiif_topics.pdf.

In [ ]:
n = np.arange(15)

In [ ]:
n

In [ ]:
lags = 3

In [ ]:
g = TimeseriesGenerator(n, n, length=lags, batch_size=3)

In [ ]:
list(g)

In [ ]:
model = Sequential()
model.add(SimpleRNN(12, activation='relu',
                    return_sequences=True,
                    input_shape=(lags, 1)))
model.add(SimpleRNN(12, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer=Adam())

In [ ]:
model.summary()

In [ ]:
%time model.fit(g, epochs=500, verbose=False)

In [ ]:
model.predict(g).flatten()

In [ ]:
n[lags:]

In [ ]:
model.predict(g).round().flatten()

In [ ]:
model.evaluate(g)

## Long Short Term Memory (LSTM) Neural Networks

### LSTM for Estimation

In [ ]:
model = Sequential()
model.add(LSTM(12, activation='relu',
                    return_sequences=True,
                    input_shape=(lags, 1)))
model.add(LSTM(12, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer=Adam())

In [ ]:
model.summary()

In [ ]:
%time model.fit(g, epochs=500, verbose=False)

In [ ]:
n[lags:]

In [ ]:
model.predict(g).round().flatten()

In [ ]:
model.evaluate(g)

### Categorical Classification

In [ ]:
list(g)

In [ ]:
len(set(n))

In [ ]:
class CategoricalTimeseriesGenerator(TimeseriesGenerator):
    def __getitem__(self, index):
        X, y = super().__getitem__(index)
        y_categorical = to_categorical(y, num_classes=len(set(n)))
        return X, y_categorical

In [ ]:
g_cat = CategoricalTimeseriesGenerator(n, n, length=lags, batch_size=3)

In [ ]:
#list(g_cat)[:2]

In [ ]:
model = Sequential()
model.add(LSTM(12, activation='relu',
                    return_sequences=True,
                    input_shape=(lags, 1)))
model.add(LSTM(12, activation='relu'))
model.add(Dense(len(set(n)), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [ ]:
%time model.fit(g_cat, epochs=250, verbose=False)

In [ ]:
model.predict(g_cat)[:2]

In [ ]:
np.argmax(model.predict(g_cat), axis=1)

In [ ]:
model.evaluate(g_cat)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>